In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch

import pickle

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample

c:\Users\Revalda Putawara\.conda\envs\bnntest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tqdm import tqdm

In [4]:
PRIORCHOICE = 'Bayesian' #options : 'Bayesian', 'Laplace'
mu = 0
sigma = 10.
LR = 1e-3

In [5]:
from utils.model import BayesianCNNSingleFCCustom, LaplaceBayesianCNNSingleFCCustom


In [6]:
device = torch.device("cuda")

In [7]:
num_classes = 10

In [8]:
if PRIORCHOICE == 'Bayesian':
    bayesian_model = BayesianCNNSingleFCCustom(num_classes=num_classes,
                                      mu=mu,
                                      sigma=sigma,
                                      device=device).to(device)
elif PRIORCHOICE == 'Laplace':
    bayesian_model = LaplaceBayesianCNNSingleFCCustom(num_classes=num_classes,
                                             mu=mu,
                                             b=b,
                                             device=device).to(device)

In [ ]:
#from utils.function import load_data,train_svi_with_stats, plot_training_results_with_stats
# from utils.function import load_data

In [10]:
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.optim import Adam

In [11]:
guide = AutoDiagonalNormal(bayesian_model)
#guide = AutoLowRankMultivariateNormal(bayesian_model, rank=10)

# 2. Optimizer and SVI - increase learning rate for better convergence
optimizer = Adam({"lr": LR})  # Increased from 1e-4 to 1e-3
svi = pyro.infer.SVI(model=bayesian_model,
                     guide=guide,
                     optim=optimizer,
                     loss=pyro.infer.Trace_ELBO())

In [15]:
from utils.function import load_data, train_svi_with_stats, plot_training_results_with_stats

In [16]:
pyro.clear_param_store()

# Ensure model and guide are on the correct device
bayesian_model.to(device)
guide.to(device)

train_loader, test_loader = load_data(batch_size=54)

# Train with statistics recording
losses, accuracies, accuracy_epochs, weight_stats, bias_stats = train_svi_with_stats(
    bayesian_model, guide, svi, train_loader, num_epochs=10
)

Calculating accuracy for epoch 1: 100%|██████████| 400/400 [00:04<00:00, 87.44it/s]


Epoch 1 - ELBO Loss: 24599198.4619, Train Accuracy: 36.77%


Epoch 2/10: 100%|██████████| 400/400 [00:12<00:00, 30.81it/s]


Epoch 2 - ELBO Loss: 12880669.3466


Epoch 3/10: 100%|██████████| 400/400 [00:13<00:00, 30.47it/s]


...

Epoch 8 - ELBO Loss: 4437683.8374


Epoch 9/10: 100%|██████████| 400/400 [00:13<00:00, 30.54it/s]


Epoch 9 - ELBO Loss: 3908273.9152


Calculating accuracy for epoch 10: 100%|██████████| 400/400 [00:04<00:00, 87.19it/s]

Epoch 10 - ELBO Loss: 3486901.3583, Train Accuracy: 57.55%
